# DEAP Dataset Preprocessing

This Notebook can be used in order to preprocess the DEAP Dataset in the following way: <br/>
1. Collect all the Data form the mat files
2. Remove the baseline from the SEEG Signals
3. Filter the EEG Signals, either with a bandpass filter, a lowpass filter or a highpass filter
4. Downsample the EEG Signals to a certain sampling rate
5. Select only certain EEG Channels and rearange them in a given order
6. Cut the EEG Signals into Windows of given length with given overlap
7. Convert the Labels from Valence, Arousal, Dominance into the Classes Negative, Neutral, Positive
8. Safe the generated Dataset
9. Generate some decent plots to compare the old signal with the new signal

## Description, Dataset Access and Citation

<b>For a full description and access to the Dataset please see:</b> <br/>
url: http://www.eecs.qmul.ac.uk/mmv/datasets/deap/readme.html <br/>
[1] Koelstra, S.; Muhl, C.; Soleymani, M.; Lee, J.-S.; Yazdani, A.; Ebrahimi, T.; Pun, T.;Nijholt, A.; Patras, I.: Deap: A Database for Emotion Analysis using Physiological Signals. IEEE transactions on affective computing 3 (2011) 1, p. 18–31.<br/>
<b>If you find this code helpfull please cite:</b><br/>
tbd

## Hyperparameters

In [1]:
# data_dir ='../Datasets/raw/deap/data_preprocessed_python'
# output_dir = '../Datasets/full_preprocessing/'
data_dir ='E:/Databases/DEAP/data_preprocessed_python/'
output_dir = 'E:/Databases/DataPre/DEAP/full_preprocessing/'
baseline_removal_window = 3
cutoff_frequencies = [4,40]
seconds_to_use = 60
downsampling_rate = 128
channels_to_use = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
window_size = 2
window_overlap = 0
convert_labels_to_nnp = True
save_plots_to_file = True


In [2]:
import os
os.makedirs(os.path.join(output_dir, "figures"), exist_ok=True)

<b>data dir (String)</b>               - the path to the raw Dataset, e.g. "'../Datasets/raw/deap/data_preprocessed_python'" <br/>
<b>output dir (String)</b>             - the path, where you want to save the preprocessed Dataset, e.g.                                                   "myDatasets/DEAP/ <br/>
<b>baseline_removal_window (float)</b> - the timewindow, to calculate a baseline (average) from. This value gets                                             subtracted from the whole timeseries. If you don't want to use baseline_removal,                                   set it to 0 <br/>
<b>cutoff_frequencies (touple)</b>     - the cutoff frequencies for the filter, if the first value ist set to None,                                         allowpass-filtering is used, if the second value is set to None, a highpass filt                                   ist used, if both values are set, a bandpass filter is used <br/>
<b>seconds_to_use (int)</b> - The window in seconds to use from the timeseries, if for example seconds_to_use ist set to 45, only the last 45 seconds will be used. If you want to use the whole timeseries, set it to None <br/>
<b>downsampling_rate (int)</b>         - the frequency, to which the eeg signals should be downsampled. If you don't want to downsample the signal, set it to 0 <br/>
<b>channels_to_use (list)</b> a list of eeg channels you want to use, if you want to use all channels from the dataset, set it to None<br/>
<b>window_size (int)</b>              - the lenght of the timewindow in seconds, the dataset should be cut into <br/>
<b>window_overlap (int)</b>            - overlap of the windows in seconds, if set to 0, the windows don't overlap <br/>
<b>convert_labels_to_nnp (boolean)</b> - wheter or not you want to convert the labels of the dataset from valence, arousal, liking, dominance to negative, neutral, positive <br/>
<b>save_plots_to_file (boolean)</b> - wheter or not you want to save the generated plots, if you choose False, the plots will be only shown in this notebook

## necessary imports

In [3]:
import numpy as np
import scipy.io as sio
from tqdm.notebook import tqdm, trange
import os
import warnings
import pickle

## Data Collection

In [4]:
sampling_rate = 128
channel_names = ['FP1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO3', 'O1', 'OZ', 'PZ', 'FP2', 'AF4', 'FZ', 'F4', 'F8', 'FC6', 'FC2', 'CZ', 'C4', 'T8', 'CP6', 'CP2', 'P4', 'P8', 'PO4', 'O2']

In [5]:
X = np.empty((1280,32,8064))
X_len = np.empty((1280,), dtype=int)
Y = np.empty((1280,4))
trial = np.empty((1280), dtype=np.int8)
subject = np.empty((1280,), dtype=np.int8)
session = np.ones((1280,), dtype=np.int8)

In [6]:
import sys
np.set_printoptions(threshold=sys.maxsize)
file_id = 0
for index, filename in tqdm(enumerate(sorted(os.listdir(data_dir)),start=1)):
    if filename.endswith(".dat"):
        temp_file = pickle.load(open(os.path.join(data_dir, filename), 'rb'), encoding='iso-8859-1')
        X_len[file_id*40:file_id*40+40] = np.ones((40,))*temp_file['data'].shape[2]
        X[file_id*40:file_id*40+40,:,:] = temp_file['data'][:,:32,:]
        Y[file_id*40:file_id*40+40,:] = temp_file['labels']
        subject[file_id*40:file_id*40+40] = np.ones((40,), dtype=np.int8)* (file_id+1)
        trial[file_id*40:file_id*40+40] = np.arange(1,41)
        file_id = file_id +1

0it [00:00, ?it/s]

In [7]:
print("Shape of the Time Series Array X: " + str(X.shape))
print("Shape of the Time Series Array Y: " + str(Y.shape))
print("Unique Session Indices: " + str(np.unique(session)))
print("Unique Subject Indices: " + str(np.unique(subject)))
print("Unique Trial Indices: " + str(np.unique(trial)))
print("Minimum length of Timeseries: " + str(min(X_len)))
print("Maximum length of Timeseries: " + str(max(X_len)))

Shape of the Time Series Array X: (1280, 32, 8064)
Shape of the Time Series Array Y: (1280, 4)
Unique Session Indices: [1]
Unique Subject Indices: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32]
Unique Trial Indices: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40]
Minimum length of Timeseries: 8064
Maximum length of Timeseries: 8064


In [8]:
# X_raw is later used for plotting, if you don't want to see the plots, you can uncomment this line
X_raw = X.copy()

## Baseline-Removal

In [9]:
if not(baseline_removal_window==0):
    baseline_datapoints = baseline_removal_window * sampling_rate
    baseline = X[:,:,:baseline_datapoints].sum(2) / baseline_datapoints
    for timestep in trange(X.shape[2]):
        X[:,:,timestep] = X[:,:,timestep] - baseline

  0%|          | 0/8064 [00:00<?, ?it/s]

## Filtering (Bandpass or Highpass or Lowpass)

In [10]:
from scipy.signal import butter, sosfilt, sosfreqz

def butter_bandpass(lowcut, highcut, fs, btype='band', order=5):
        nyq = 0.5 * fs
        if btype == 'bandpass':
            low = lowcut / nyq
            high = highcut / nyq
            sos = butter(order, [low, high], analog=False, btype='bandpass', output='sos')
        elif btype == 'highpass':
            low = lowcut / nyq
            sos = butter(order, low, analog=False, btype='highpass', output='sos')
        elif btype == 'lowpass':
            high = highcut / nyq
            sos = butter(order, high, analog=False, btype='lowpass', output='sos')
        return sos

def butter_bandpass_filter(X, lowcut, highcut, fs, btype='bandpass', order=5):
        sos = butter_bandpass(lowcut, highcut, fs, btype=btype, order=order)
        X = sosfilt(sos, X)
        return X

In [11]:
if not(cutoff_frequencies[0] == None):
    if not(cutoff_frequencies[1] == None):
        btype='bandpass'
    else:
        btype='highpass'
elif not (cutoff_frequencies[1] == None):
        btype='lowpass'

for experiment_id in trange(X.shape[0]):
    for channel_id in range(X.shape[1]):
        X[experiment_id, channel_id, :] = butter_bandpass_filter(
                                                        X[experiment_id, channel_id, :],
                                                        cutoff_frequencies[0],
                                                        cutoff_frequencies[1],
                                                        sampling_rate,
                                                        btype=btype,
                                                        order=5)

  0%|          | 0/1280 [00:00<?, ?it/s]

In [12]:
print(f"x: {X.shape}")

x: (1280, 32, 8064)


## Use only the last n seconds of the timeseries
(As determined with the hyperparameter seconds_to_use)

In [13]:
if not(seconds_to_use == None):
    num_sample_points_to_use = seconds_to_use * sampling_rate
    X_selected = np.zeros((X.shape[0], X.shape[1], num_sample_points_to_use))
    for exp_id in trange(len(X_len)):
        X_selected[exp_id,:,:] = X[exp_id,:,X_len[exp_id]-num_sample_points_to_use:X_len[exp_id]]
    X = X_selected

  0%|          | 0/1280 [00:00<?, ?it/s]

In [14]:
print(f"x: {X.shape}")

x: (1280, 32, 7680)


## Downsampling

In [15]:
from scipy.signal import resample

In [16]:
if not(downsampling_rate == 0) and not(downsampling_rate == sampling_rate):
    new_length = int(X.shape[2] / sampling_rate * downsampling_rate)
    X_downsampled = np.zeros((X.shape[0], X.shape[1], new_length))
    for experiment_id in trange(X.shape[0]):
        for channel_id in range(X.shape[1]):
            X_downsampled[experiment_id, channel_id, :] = resample(X[experiment_id, channel_id, :], new_length)
    X = X_downsampled

## Select certain channels

In [17]:
if channels_to_use == None:
    channels_to_use = channel_names
print(channels_to_use)

['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']


In [18]:
channel_index_list = list()
for i in range(len(channels_to_use)):
    if channels_to_use[i] in channel_names:
        channel_index_list.append(channel_names.index(channels_to_use[i]))
    else:
        warnings.warn(' Channel ' + channels_to_use[i] +' could not be found in the list of actual channels')
print(channel_index_list)
print(len(channel_index_list))

[1, 3, 2, 4, 7, 11, 13, 31, 29, 25, 21, 19, 20, 17]
14


In [19]:
X_selected_channels = np.zeros((X.shape[0], len(channels_to_use), X.shape[2]))
for channel in trange(len(channel_index_list)):
    X_selected_channels[:,channel,:] = X[:,channel_index_list[channel],:]
X = X_selected_channels

  0%|          | 0/14 [00:00<?, ?it/s]

In [20]:
print(f"x: {X.shape}")

x: (1280, 14, 7680)


## Cut into windows

In [21]:
# 计算每个窗口的点数和每个窗口的重叠点数
num_points_per_window = window_size * downsampling_rate
num_points_overlap = window_overlap * downsampling_rate

# 计算窗口滑动的步长
stride = num_points_per_window - num_points_overlap

# 初始化起始和结束索引列表
start_index = [0]
end_index = [num_points_per_window]

# 初始化每个实验的窗口数
num_windows_per_exp = 1

# 计算每个实验的窗口数以及相应的起始和结束索引
while(end_index[-1] + stride < X.shape[2]):
    num_windows_per_exp += 1
    start_index.append(start_index[-1] + stride)
    end_index.append(end_index[-1] + stride)

# 初始化切割后的数据数组
X_cut = np.zeros((num_windows_per_exp * X.shape[0], X.shape[1], num_points_per_window))
Y_cut = np.zeros((num_windows_per_exp * X.shape[0], 4))
session_cut = np.zeros(num_windows_per_exp * X.shape[0])
subject_cut = np.zeros(num_windows_per_exp * X.shape[0])
trial_cut = np.zeros(num_windows_per_exp * X.shape[0])

# 遍历每个实验
for exp_id in trange(X.shape[0]):
    # 遍历每个窗口
    for window_id in range(len(start_index)):
        # 根据窗口的起始和结束索引切割数据
        X_cut[exp_id * num_windows_per_exp + window_id, :, :] = X[exp_id, :, start_index[window_id]:end_index[window_id]]
        # 复制标签、会话、受试者和试验信息
        Y_cut[exp_id * num_windows_per_exp + window_id, :] = Y[exp_id, :]
        session_cut[exp_id * num_windows_per_exp + window_id] = session[exp_id]
        subject_cut[exp_id * num_windows_per_exp + window_id] = subject[exp_id]
        trial_cut[exp_id * num_windows_per_exp + window_id] = trial[exp_id]

# 更新原始数据为切割后的数据
X = X_cut
Y = Y_cut
session = session_cut
subject = subject_cut
trial = trial_cut


  0%|          | 0/1280 [00:00<?, ?it/s]

In [25]:
print(X.shape)
print(f"session shape: {np.array(session).shape}")
print(f"session: {np.unique(session)}")
print(f"subject: {np.unique(subject)}")
print(f"trial: {np.unique(trial)}")

(37120, 14, 256)
session shape: (37120,)
session: [1.]
subject: [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32.]
trial: [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36.
 37. 38. 39. 40.]


## Label Conversion
The Labels get converted from Valence Arousal Dominance Liking into the Classes Negative, Neutral, Positive using a <b>K-Means Clustering </b>

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [ ]:
def sort_centeroids(centeroids):
    # calculates the distance between given centeroids
    # and the corners bottom-right, bottom-left, top-left, top-right
    # and returns them in the order that the point closest to the br corner is returned first,
    # the one closest to the bl second,
    # the one clostest to the tl third,
    # and the one closest to the top right fourth
    
    distance_br = np.zeros(centeroids.shape[0])
    distance_bl = np.zeros(centeroids.shape[0])
    distance_tl = np.zeros(centeroids.shape[0])
    distance_tr = np.zeros(centeroids.shape[0])
    
    for i in range(centeroids.shape[0]):
        distance_br[i] = abs((1-centeroids[i,0])**2 + (-1-centeroids[i,1])**2)
        distance_bl[i] = abs((-1-centeroids[i,0])**2 + (-1-centeroids[i,1])**2)
        distance_tl[i] = abs((-1-centeroids[i,0])**2 + (1-centeroids[i,1])**2)
        distance_tr[i] = abs((1-centeroids[i,0])**2 + (1-centeroids[i,1])**2)
    
    br_idx = np.argmin(distance_br)
    bl_idx = np.argmin(distance_bl)
    tl_idx = np.argmin(distance_tl)
    tr_idx = np.argmin(distance_tr)
    
    return br_idx, bl_idx, tl_idx, tr_idx

In [ ]:
scaler = MinMaxScaler(feature_range=(-1,1))
valence = scaler.fit_transform(Y[:,0].reshape(-1,1))
arousal = scaler.fit_transform(Y[:,1].reshape(-1,1))
datapoints = np.concatenate((valence,arousal),1)

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=7).fit(datapoints)

In [ ]:
sad_label, fear_label, neutral_label, happy_label = sort_centeroids(kmeans.cluster_centers_)

In [ ]:
plt.scatter(valence,arousal, c=kmeans.labels_, edgecolors='none')
plt.xlabel('Valence')
plt.ylabel('Arousal')
plt.title('DEAP - NNP Label via K-Means')
plt.show()
print('#Fear: %i - #Sad: %i - #Neutral: %i - Happy: %i'
      %(np.count_nonzero(kmeans.labels_ == fear_label),
        np.count_nonzero(kmeans.labels_ == sad_label),
        np.count_nonzero(kmeans.labels_ == neutral_label),
        np.count_nonzero(kmeans.labels_ == happy_label)))

In [ ]:
idx_sad = np.where(kmeans.labels_==sad_label)
idx_fear = np.where(kmeans.labels_==fear_label)
idx_neutral = np.where(kmeans.labels_==neutral_label)
idx_happy = np.where(kmeans.labels_==happy_label)

In [ ]:
Y_nnp = np.zeros(Y.shape[0],)
Y_nnp[idx_sad] = -1
Y_nnp[idx_fear] = -1
Y_nnp[idx_neutral] = 0
Y_nnp[idx_happy] = 1

In [ ]:
if convert_labels_to_nnp:
    Y = Y_nnp

In [ ]:
print(f"X: {X.shape}")
print(f"y: {Y.shape}")

## Safe Dataset

Please note: This step can take up to several minutes

In [ ]:

np.savez_compressed(
                    output_dir +'/DEAP.npz',
                    X=X,
                    Y=Y,
                    session = session,
                    subject = subject,
                    trial = trial,
                    downsampling_rate = downsampling_rate,
                    channel_names = channels_to_use,
                    window_size=window_size,
                    window_overlap = window_overlap,
                    cutoff_frequencies = cutoff_frequencies,
                    baseline_removal_window = baseline_removal_window,
                    seconds_to_use = seconds_to_use
                    )
print('Saved File')

## Plots

In [ ]:
def perform_FFT(X, sampling_rate):
    n = len(X) # length of the signal
    k = np.arange(n)
    T = n/sampling_rate
    frq = k/T # two sides frequency range
    frq = frq[:len(frq)//2] # one side frequency range

    X_FT = np.fft.fft(X)/n # dft and normalization
    X_FT = X_FT[:n//2]
    return X_FT, frq

In [ ]:
channel_id_to_plot = 1

In [ ]:
raw_ts = X_raw[0,channel_names.index(channels_to_use[channel_id_to_plot]),0:sampling_rate*window_size]
preprocessed_ts = X[0,channel_id_to_plot,:]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(np.arange(0,sampling_rate*window_size),raw_ts, label="Raw Signal")
plt.plot(np.arange(0,downsampling_rate*window_size)/downsampling_rate*sampling_rate,preprocessed_ts, label="Preprocessed Signal")
plt.title("Comparison of an Exemplary Time Series")
plt.xticks([0,sampling_rate,2*sampling_rate],[0,1,2])
plt.xlabel("Time t [s]")
plt.ylabel("Voltage U [mV]")
plt.legend()
if save_plots_to_file:
    plt.savefig(output_dir+'figures/DEAP_Timespace.png', facecolor="white")
else:
    plt.show()

In [ ]:
raw_fs, raw_frq = perform_FFT(raw_ts, sampling_rate)
preprocessed_fs, preprocessed_frq = perform_FFT(preprocessed_ts, downsampling_rate)

In [ ]:
plt.plot(raw_frq, abs(raw_fs), label="Raw Signal")
plt.plot(preprocessed_frq, abs(preprocessed_fs), label="Preprocessed Signal")
plt.title("Comparison of an Exemplary Spectrum")
plt.xlabel("Frequency f [Hz]")
plt.ylabel("Amplitude |X(f)|")
plt.legend()
if save_plots_to_file:
    plt.savefig(output_dir+'figures/DEAP_Frequencyspace.png', facecolor="white")
else:
    plt.show()

# 数据集含义

在机器学习和深度学习中，数据通常被分割成训练集、验证集和测试集。这些数据集分别用于训练模型、调参和验证模型、以及最终评估模型性能。以下是各个数据集的含义：

1. 训练集 (Training Set):

    - X_train: 训练集的特征数据，输入数据集。
    - Y_train: 训练集的标签数据，目标输出。
    - S_train: 训练集的受试者ID，表示每个样本对应的受试者。
    - P_train: 训练集的试验阶段或试次，表示每个样本对应的实验阶段或试次。

2. 验证集 (Validation Set):

    - X_val: 验证集的特征数据，用于调参和验证模型。
    - Y_val: 验证集的标签数据。
    - S_val: 验证集的受试者ID。
    - P_val: 验证集的试验阶段或试次。

3. 测试集 (Test Set):

    - X_test: 测试集的特征数据，用于最终评估模型性能。
    - Y_test: 测试集的标签数据。
    - S_test: 测试集的受试者ID。
    - P_test: 测试集的试验阶段或试次。

各个数据集的具体含义如下：

- X_train, X_val, X_test: 这些是特征矩阵（例如EEG数据），每一行表示一个样本，每一列表示一个特征。
- Y_train, Y_val, Y_test: 这些是标签向量，表示每个样本对应的类别或目标值。
- S_train, S_val, S_test: 这些是受试者ID，表示每个样本对应的受试者编号，有助于在跨受试者的实验中进行分析。
- P_train, P_val, P_test: 这些是试验阶段或试次，表示每个样本对应的实验阶段或试次编号，有助于在跨试次的实验中进行分析。

In [ ]:
import numpy as np
import os
from sklearn.model_selection import train_test_split

# 加载DEAP数据集
filename = "E:/Databases/DataPre/DEAP/full_preprocessing/DEAP.npz"
output_dir = "E:/Databases/DataPre/DEAP/full_preprocessing/"
version = 'bs2'
data = np.load(filename)
X = data['X']
Y = data['Y']
subjects = data['subject']
sessions = data['session']
trials = data['trial']

# 分割数据集
X_train, X_temp, Y_train, Y_temp, S_train, S_temp, P_train, P_temp = train_test_split(
    X, Y, subjects, sessions, test_size=0.4, random_state=42)
X_val, X_test, Y_val, Y_test, S_val, S_test, P_val, P_test = train_test_split(
    X_temp, Y_temp, S_temp, P_temp, test_size=0.5, random_state=42)

# 调整subject和session的ID
S_train = S_train + 100 * 1
S_val = S_val + 100 * 2
S_test = S_test + 100 * 3

# 保存数据集
np.savez(
        os.path.join(output_dir, version, 'DEAP.npz'),
         X_train=X_train, X_val=X_val, X_test=X_test,
         Y_train=Y_train, Y_val=Y_val, Y_test=Y_test,
         S_train=S_train, S_val=S_val, S_test=S_test,
         P_train=P_train, P_val=P_val, P_test=P_test)
